In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

### Задача 1 
Написать функцию `get_strong(html)`, принимающую на вход html-страницу в виде длинной строки, записанной в переменную `html`, и возвращающую строчку, содержащуюся в первом теге `strong`.

Примеры см. в тестах.

**Подсказка.** Вы можете создать объект `BeautifulSoup`, передав ему строку с html в качестве параметра. Например:

```python
from bs4 import BeautifulSoup
page = BeautifulSoup("<html><body><p>Hello</p></body></html>", "html.parser")
print(page.p)
```

In [2]:
def get_strong(html):
    # init parser
    parser = BeautifulSoup(html, 'html.parser')
    # find text by need tag
    need_s = parser.find(name="strong")
    return need_s.get_text() # return need field

In [3]:
#проверка
assert get_strong("<html><body><p>Hello, <strong>World</strong>!") == "World"
html = """<html>
    <body>
        <p>
            Hello,
            <strong>
                World
            </strong>
        </p>
    </body>
</html>"""
assert get_strong(html).strip() == "World"
assert get_strong("<html><body><p>tag &lt;strong&gt; is used in HTML\n to make letters <strong>stronger</strong>") == "stronger"
assert get_strong("<html><body><strong>One\nTwo</strong></body></html>") == "One\nTwo"

### Задача 2
Написать функцию `any_news_about_harry(url)`, принимающую на вход адрес веб-страницы `url`, загружающую эту веб-страницу и проверяющую, встречается ли в ней слово `Harry` (с большой буквы). Функция должна возвращать `True`, если встречается, и `False` в противном случае. Также функция должна возвращать `False`, если страницу не удалось открыть (например, была получена ошибка *404 Not Found*.)

**Подсказка.** Чтобы загрузить страницу, нужно использовать библиотеку `requests`:

    import requests
    r = requests.get(url)

Содержимое страницы затем окажется в `r.text`. Проверить, что запрос увенчался успехом, можно так:

    if r:
        # увенчался успехом
        
В этой задаче вам не нужно использовать `BeautifulSoup`. Чтобы проверить наличие подстроки в строке, можно использовать `in`. (Например, `'ell' in "Hello"` вернёт True.)

In [4]:
def any_news_about_harry(url):
    try:
        # make request
        r = requests.get(url)
        # check condition
        if 'Harry' in r.text:
            return True
        else:
            return False
    except:
        return False

In [5]:
# проверка
assert any_news_about_harry("https://en.wikipedia.org/w/index.php?title=J._K._Rowling&oldid=694008857")
assert any_news_about_harry("https://en.wikipedia.org/w/index.php?title=Star_Wars&oldid=694701430")
assert not any_news_about_harry("https://en.wikipedia.org/w/index.php?title=Darth_Vader&oldid=694617684")

### Задача 3
Для вставки картинок в HTML используется тег `<img>`, содержащий параметр `src` — адрес файла с картинкой. Например, `<img src="https://upload.wikimedia.org/wikipedia/commons/b/bd/Struthio_camelus_portrait_Whipsnade_Zoo.jpg"/>`. Написать функцию `all_images_src(html)`, принимающую на вход длинную строчку с HTML-документом, а возвращающую список адресов всех картинок, встречающихся в этом документе (в том порядке, в котором они встречаются в документе).

**Подсказка.** Для обращения к атрибутам тега нужно использовать квадратные скобки, как если бы тег был словарём.

In [6]:
def all_images_src(html):
    # init parser
    parser = BeautifulSoup(html, 'html.parser')
    # find text by need tag
    need_s = parser.find_all({'img'})
    # get all images
    imgs = [need_s[i]['src'] for i in range(len(need_s))]
    return imgs

In [7]:
# проверка
assert all_images_src('<html><body><img src="https://upload.wikimedia.org/wikipedia/commons/b/bd/Struthio_camelus_portrait_Whipsnade_Zoo.jpg"/>') == ["https://upload.wikimedia.org/wikipedia/commons/b/bd/Struthio_camelus_portrait_Whipsnade_Zoo.jpg"]
assert all_images_src( ('<html><body><IMG src="test.jpg">\n'
                        '<p>Some text\n'
                        '<img SRC=\'well.png\'>\n'
                        '</p></body></html>') ) == ["test.jpg", "well.png"]
assert all_images_src('<html><body><p><a href="link.html">'
                      '<img alt="Just a test image" src="this is a test.jpg"><ul>' + "\n"
                      .join("<li><img src='img%04i.png'></li>" % i for i in range(1000)) + 
                      "</ul></p></body></html>"
                     ) == ['this is a test.jpg'] + ['img%04i.png' % i for i in range(1000)]

### Задача 4

Написать функцию `city_tz(name)`, принимающую на вход название города и возвращающую строку, содержащую часовой пояс, действующий в этом городе (например, `'UTC+3'`), согласно данным русской Википедии. Если такого города Википедия не знает, или если у города не указан часовой пояс `None`.

Предполагается, что вы будете решать эту задачу, обрабатывая HTML-код веб-страницы, а не исходный код статей, и не будете пользоваться сторонними библиотеками (кроме `requests`, `BeautifulSoup`).

**Подсказка.** Можно передать адрес страницы напрямую в `requests.get`, например `https://ru.wikipedia.org/wiki/Санкт-Петербург` [ссылка](https://ru.wikipedia.org/wiki/Санкт-Петербург).

In [8]:
import re

def city_tz(name):
    # city wiki template
    wiki_template = 'https://ru.wikipedia.org/wiki/{}'
    # get html by wiki url
    req = requests.get(wiki_template.format(name))

    # init parser
    parser = BeautifulSoup(req.text, 'html.parser')
    # find pattenr with '/wiki/UTC'
    # <span data-wikidata-property-id="P421" class="no-wikidata"><a href="/wiki/UTC%2B3:00" title="UTC+3:00">UTC+3:00</a></span>
    need_s = parser.find('a', href=re.compile("/wiki/UTC%"), title=re.compile("UTC"))
    # if search data not exist return None 
    try:
        return need_s.get_text()
    except:
        return None

In [9]:
#Проверка
res = [
    ("Абакан", "UTC+7:00"),
    ("Анадырь", "UTC+12:00"),
    ("Киров (Кировская область)", "UTC+3:00"),
    ("Южно-Сахалинск", "UTC+11:00"),
    ("Усть-Каменоустюгск", None),
]
for city, site in res:
    assert city_tz(city) == site, (site, city_tz(city))

### Задача 5
Написать функцию `get_all_headings(url)`, принимающую на вход адрес страницы в Википедии и возвращающую список, состоящий из названий разделов статьи (в порядке появления в статье). Если такой страницы не существует, функция должна вернуть список, состоящей из строки `"Not found"`.

**Подсказка.** С помощью функции вашего браузера *inspect element* или аналогичной, исследуйте, в каких тегах и с какими классами находятся искомые заголовки. Не во всех страницах есть содержание! Например, ваш код должен корректно обрабатывать [эту страницу](https://ru.wikipedia.org/w/index.php?title=User%3AIlya_Voyager%2Fsandbox%2Fh2test&oldid=75055744).

In [10]:
def get_all_headings(url):
    # get html by wiki url
    req = requests.get(url)

    # init parser
    parser = BeautifulSoup(req.text, 'html.parser')
    # find pattenr 
    need_s = parser.find_all('h2')
    s = []
    for i in range(len(need_s)):
        v = str(need_s[i]).split('span class="mw-headline" id="')[-1].split('">')[0]
        if '<h2' not in v:
            s += [v.replace('_', ' ')]
            
    # if search data not exist return None 
    if len(s) != 0:
        return s
    else:
        return ["Not found"]

In [11]:
#проверка
from urllib.parse import urlencode
entrypoint = "https://ru.wikipedia.org/w/index.php?"
def mkurl(title, oldid):
    return entrypoint+urlencode(dict(title=title, oldid=oldid))
assert get_all_headings(mkurl("Северовирджинская кампания",75043192)) == ['Предыстория',
                                                                          'Силы сторон',
                                                                          'Сражения',
                                                                          'Последствия',
                                                                          'Примечания',
                                                                          'Литература',
                                                                          'Ссылки']

assert get_all_headings(mkurl('User:Ilya_Voyager/sandbox/h2test',"75055744")) == ['Заголовок', 'Ещё один заголовок', 'Третий заголовок']
assert get_all_headings(mkurl('User:Ilya_Voyager/This Page Will Never Exist', "")) == ["Not found"]
del urlencode, mkurl

### Задача 6
Необходимо загрузить таблицу со списком *действующих* станций Петербургского метрополитена со [страницы](https://ru.wikipedia.org/wiki/Список_станций_Петербургского_метрополитена) в датафрейм `subway_spb`:
* Необходимые столбцы: Название станции, Дата открытия, Глубина, Тип станции
* Преобразовать дату открытия в `datetime`

На основании полученного датафрейма найти:
* Самую старую станцию и её возраст в годах (с учётом неполных лет)
* Самую глубокую станцию
* Распределение количества станций по типу (в виде `pd.DataFrame` или `pd.Series`)
* Самый распространенный тип станции

**Подсказка** Воспольщуйтесь функцией `pd.read_html()`. Если не получается передать в функцию `url` напрямую, сначала загрузите страницу с помощью `requests.get()`, а после передайте результат запроса в виде текста в `pd.read_html()`.

In [4]:
import pandas as pd
import numpy as np
import datetime
import re

In [21]:
url = 'https://ru.wikipedia.org/wiki/Список_станций_Петербургского_метрополитена'
req = requests.get(url)

# init parser
main_parser = BeautifulSoup(req.text, 'html.parser')
st_info = main_parser.find_all('td')
# st_names = main_parser.find_all()('a', title=re.compile('станция метро'))

stations = []
dates_open = []
st_type = []
depths = []
    
# find station names
for i in range(1, len(st_info), 8):
    stations += [st_info[i].get_text()]

# find open date
for i in range(2, len(st_info), 8):
    dates_open += [st_info[i].get_text()]
    
# find type station 
for i in range(3, len(st_info), 8):
    st_type += [st_info[i].get_text()]
    
# find depth station 
for i in range(4, len(st_info), 8):
    depths += [st_info[i].get_text()]

In [25]:
# create dataframe
df = pd.DataFrame({'station': stations[:len(dates_open)], 'dates_open': dates_open, 'type': st_type, 'depth': depths})

# make depth int
df.depth = pd.to_numeric(df.depth, errors='coerce')
df = df.dropna()
df.depth = df.depth.astype(int)

# filter type station
df.type = df.type.str.split('\n').str[0]

In [27]:
# date open to date format
def change_ru_month(x):
    month_dict = {'января': '01',
                  'февраля': '02', 
                  'марта': '03',
                  'апреля': '04', 
                  'мая': '05',
                  'июня': '06',
                  'июля': '07',
                  'августа': '08',
                  'сентября': '09',
                  'октября': '10',
                  'ноября': '11',
                  'декабря': '12'}
    s = x.split(' ')
    if len(s) != 3:
        return np.nan
    else:
        s[1] = month_dict[s[1]]
        return '-'.join(s)
    

df.dates_open = df.dates_open.apply(change_ru_month)
df = df.dropna()
df.dates_open = pd.to_datetime(df.dates_open)

In [28]:
# всего станций в метро спб -- 72. как и у нас
df.shape[0]

72

In [29]:
df.dates_open.min()

Timestamp('1955-11-15 00:00:00')

In [30]:
df

,station,dates_open,type,depth
0,Девяткино до 1 июля 1992 «Комсомольская»\n,1978-12-29,крытая наземная,0
1,Гражданский проспект\n,1978-12-29,колонная,64
2,Академическая\n,1975-12-31,колонная,64
3,Политехническая\n,1975-12-31,односводчатая,65
4,Площадь Мужества\n,1975-12-31,односводчатая,67
...,...,...,...,...
67,Бухарестская\n,2012-12-28,пилонная,65
68,Международная\n,2012-12-28,колонно-стеновая,65
69,Проспект Славы\n,2019-03-10,пилонная,59
70,Дунайская\n,2019-03-10,колонная многопролётная мелкого заложения,17


На основании полученного датафрейма найти:

* Самую старую станцию и её возраст в годах (с учётом неполных лет)
* Самую глубокую станцию
* Распределение количества станций по типу (в виде pd.DataFrame или pd.Series)
* Самый распространенный тип станции

In [31]:
# Самую старую станцию и её возраст в годах (с учётом неполных лет)
df['age'] = (datetime.datetime.now() - df.dates_open)
df[df.age == df.age.max()]

,station,dates_open,type,depth,age
9,Площадь Восстания\n,1955-11-15,пилонная,58,24508 days 10:11:46.364778
10,Владимирская\n,1955-11-15,пилонная с укороченным центральным залом,55,24508 days 10:11:46.364778
12,Технологический институт\n,1955-11-15,колонная,60,24508 days 10:11:46.364778
13,Балтийская\n,1955-11-15,колонная,57,24508 days 10:11:46.364778
14,Нарвская\n,1955-11-15,пилонная,52,24508 days 10:11:46.364778
15,Кировский завод\n,1955-11-15,колонная,50,24508 days 10:11:46.364778
16,Автово\n,1955-11-15,колонная трёхпролётная мелкого заложения,12,24508 days 10:11:46.364778


In [32]:
# Самую глубокую станцию
df[df.depth == df.depth.max()]

,station,dates_open,type,depth,age
62,Адмиралтейская\n,2011-12-28,колонно-стеновая,86,4011 days 10:11:46.364778


In [160]:
# Распределение количества станций по типу (в виде pd.DataFrame или pd.Series)
df.type.value_counts()

односводчатая                                14
пилонная                                     11
колонная                                     10
закрытого типа                               10
колонно-стеновая                              8
пилонная с укороченным центральным залом      7
крытая наземная                               5
колонная трёхпролётная мелкого заложения      3
колонная многопролётная мелкого заложения     3
двухъярусная пересадочная односводчатая       1
Name: type, dtype: int64

In [33]:
# Самый распространенный тип станции
df.type.value_counts().index[df.type.value_counts().argmax()]

'односводчатая'